In [3]:
import malariagen_data
import allel
import pandas as pd

In [4]:
ag3 = malariagen_data.Ag3(
    "simplecache::gs://vo_agam_release",
    pre=True, 
    debug=False,
    simplecache=dict(cache_storage="../gcs_cache")
)
ag3

<MalariaGEN Ag3 API>
Storage URL             : simplecache::gs://vo_agam_release
Data releases available : 3.0, 3.1, 3.2, 3.3, 3.4, 3.5
Results cache           : None
Cohorts analysis        : 20211101
Species analysis        : aim_20200422
Site filters analysis   : dt_20200416
Software version        : malariagen_data 3.0.0
Client location         : England, GB
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

In [5]:
ag3.snp_variants(region="3L")

<xarray.Dataset>
Dimensions:                             (variants: 40758473, alleles: 4)
Coordinates:
    variant_position                    (variants) int32 dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_contig                      (variants) uint8 dask.array<chunksize=(524288,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(524288, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [6]:
ag3.snp_calls(region="3L", sample_sets="AG1000G-FR")

<xarray.Dataset>
Dimensions:                             (variants: 40758473, alleles: 4, samples: 23, ploidy: 2)
Coordinates:
    variant_position                    (variants) int32 dask.array<chunksize=(524288,), meta=np.ndarray>
    variant_contig                      (variants) uint8 dask.array<chunksize=(524288,), meta=np.ndarray>
    sample_id                           (samples) <U24 dask.array<chunksize=(23,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele                      (variants, alleles) |S1 dask.array<chunksize=(524288, 1), meta=np.ndarray>
    variant_filter_pass_gamb_colu_arab  (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_gamb_colu       (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    variant_filter_pass_arab            (variants) bool dask.array<chunksize=(300000,), meta=np.ndarray>
    call_genotype                       (variants, samples, ploidy) int8 dask.array<chunksize=(300000, 23, 2), meta=np.ndarray>
    call_GQ                             (variants, samples) int16 dask.array<chunksize=(300000, 23), meta=np.ndarray>
    call_MQ                             (variants, samples) int16 dask.array<chunksize=(300000, 23), meta=np.ndarray>
    call_AD                             (variants, samples, alleles) int16 dask.array<chunksize=(300000, 23, 4), meta=np.ndarray>
    call_genotype_mask                  (variants, samples, ploidy) bool dask.array<chunksize=(300000, 23, 2), meta=np.ndarray>
Attributes:
    contigs:  ('2R', '2L', '3R', '3L', 'X')

In [7]:
def _snps_track_data(
    region, 
    sample_sets=None, 
    sample_query=None, 
    site_mask=None):
    self = ag3
    
    ac = allel.AlleleCountsArray(self.snp_allele_counts(
        region=region, 
        sample_sets=sample_sets,
        sample_query=sample_query,
    ))
    an = ac.sum(axis=1)
    is_seg = ac.is_segregating()
    is_var = ac.is_variant()
    allelism = ac.allelism()
    
    ds_sites = self.snp_variants(
        region=region,
    ).compute()
    
    # build a dataframe
    pos = ds_sites["variant_position"].values
    alleles = ds_sites["variant_allele"].values.astype("U")
    cols = {
        "pos": pos,
        "allele_0": alleles[:, 0],
        "allele_1": alleles[:, 1],
        "allele_2": alleles[:, 2],
        "allele_3": alleles[:, 3],
        "ac_0": ac[:, 0],
        "ac_1": ac[:, 1],
        "ac_2": ac[:, 2],
        "ac_3": ac[:, 3],
        "an": an,
        "is_seg": is_seg,
        "is_var": is_var,
        "allelism": allelism,
    }
    
    return pd.DataFrame(cols)

_snps_track_data(
    region="3L:1_000_000-1_001_000", 
    sample_sets="AG1000G-FR"
)    

Compute SNP allele counts:   0%|          | 0/2 [00:00<?, ?it/s]

,pos,allele_0,allele_1,allele_2,allele_3,ac_0,ac_1,ac_2,ac_3,an,is_seg,is_var,allelism
0,1000483,T,A,C,G,0,0,0,0,0,False,False,0
1,1000484,G,A,C,T,0,0,0,0,0,False,False,0
2,1000485,C,A,T,G,0,0,0,0,0,False,False,0
3,1000486,G,A,C,T,0,0,0,0,0,False,False,0
4,1000487,T,A,C,G,0,0,0,0,0,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,1000996,G,A,C,T,44,0,0,0,44,False,False,1
514,1000997,C,A,T,G,44,0,0,0,44,False,False,1
515,1000998,G,A,C,T,46,0,0,0,46,False,False,1
516,1000999,A,C,T,G,46,0,0,0,46,False,False,1
